# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans

In [3]:
spark = SparkSession.builder.appName('ClusterConsulting').getOrCreate()

In [4]:
data = spark.read.csv("/FileStore/tables/hack_data.csv",header=True,inferSchema=True)

In [5]:
data.head()

In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used','Servers_Corrupted','Pages_Corrupted','WPM_Typing_Speed'],outputCol='features')

In [8]:
data_vec = assembler.transform(data)

In [9]:
from pyspark.ml.feature import StandardScaler

In [10]:
scaler = StandardScaler(inputCol='features',outputCol='scaledFeatures',withStd=True,withMean=False)

In [11]:
scalerModel = scaler.fit(data_vec)

In [12]:
data_scaledVec = scalerModel.transform(data_vec)

In [13]:
data_scaledVec.head(1)

In [14]:
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
model2 = kmeans2.fit(data_scaledVec)
model3 = kmeans3.fit(data_scaledVec)

In [15]:
wssse2 = model2.computeCost(data_scaledVec)
wssse3 = model3.computeCost(data_scaledVec)
print("2 Clusters Within Set Sum of Squared Errors = " + str(wssse2))
print("3 Clusters Within Set Sum of Squared Errors = " + str(wssse3))

In [16]:
model2.transform(data_scaledVec).groupBy('prediction').count().show()

In [17]:
model3.transform(data_scaledVec).groupBy('prediction').count().show()

In [18]:
#Hence two group of hackers.